In [ ]:
import cv2
import numpy as np
import time
import threading
import math
import random
import pynput.keyboard as pkeyboard
from collections import Counter
from IPython.display import clear_output

import sys, os
sys.path.append("../x11-window-interactor")
from x11_interactor import X11WindowInteractor
sys.path.append("../scale-invariant-template-matching")
from template_matching import CannyEdgeMatcher

In [ ]:
# Initialize mouse and keyboard controllers
interactor = X11WindowInteractor()

# Canny matcher
matcher = CannyEdgeMatcher(canny_low=25, canny_high=50, match_threshold=0.1)

# Global variable to control the start and stop of the script
script_running = False

## HOH Presurge (ROI)
## 4K - 150%
# hoh_presurge_roi_data = (1600, 200, 200, 200)
## 1080P - scaled
# hoh_presurge_roi_data = (1265, 190, 60, 60)
## 1080P - standard
hoh_presurge_roi_data = (845, 53, 46, 41)

## Dark Portal postsurge
## 4K - 150%
# dark_portal_postsurge_data = (1650, 815, 60, 60)
## 1080P - scaled
# dark_portal_postsurge_data = (1270, 605, 20, 20)
## 1080P - standard
dark_portal_postsurge_data = (841, 390, 27, 26)

# Stores scales for all template matches
template_scales = {}

In [ ]:
def find_image(template_path, screenshot, scale=None):
    global template_scales
    if template_path in template_scales:
        result_img_canny, bbox, scale, correlation, status_canny = matcher.match(template_input = template_path, target_input = screenshot, scale = template_scales[template_path])
    elif scale:
        result_img_canny, bbox, scale, correlation, status_canny = matcher.match(template_input = template_path, target_input = screenshot, scale = scale)
        if status_canny == 'Detected':
            template_scales[template_path] = scale
        else:
            return None
    else:
        result_img_canny, bbox, scale, correlation, status_canny = matcher.match(template_input = template_path, target_input = screenshot)
        if status_canny == 'Detected':
            template_scales[template_path] = scale
        else:
            return None
    return bbox

In [ ]:
def randomize_click_position(center_x, center_y, width, height, shape='rectangle', roi_diminish=2):
    if shape == 'circle':
        # Randomize within a circle
        radius = (min(width, height) // 2) // roi_diminish
        angle = np.random.uniform(0, 2 * math.pi)
        r = radius * np.sqrt(np.random.uniform(0, 1))
        click_x = int(center_x + r * math.cos(angle))
        click_y = int(center_y + r * math.sin(angle))
    else:
        # Randomize within a rectangle
        area_width = width * 0.5
        area_height = height * 0.5

        left_bound = center_x - area_width / roi_diminish
        right_bound = center_x + area_width / roi_diminish
        top_bound = center_y - area_height / roi_diminish
        bottom_bound = center_y + area_height / roi_diminish

        click_x = int(np.random.normal(center_x, area_width / 6))
        click_y = int(np.random.normal(center_y, area_height / 6))

        click_x = min(max(click_x, left_bound), right_bound)
        click_y = min(max(click_y, top_bound), bottom_bound)

    return int(click_x), int(click_y)

In [ ]:
def main_script():
    bank_boat_img = 'assets/bank_boat.png'
    load_last_bank_preset_img = 'assets/load_last_preset.png'
    haunt_on_the_hill_img = 'assets/haunt_on_the_hill.png'
    flesh_altar_presurge_img = 'assets/flesh_altar_presurge.png'
    flesh_altar_postsurge_img = 'assets/flesh_altar_postsurge.png'
    miasma_altar_presurge_img = 'assets/miasma_altar_presurge.png'
    miasma_altar_postsurge_img = 'assets/miasma_altar_postsurge.png'
    spirit_altar_postsurge_img = 'assets/spirit_altar_postsurge.png'
    
    global script_running

    cycle_count = int(input("Enter the number of runecrafting runs: "))
    rune_type = input("Enter the type of rune to craft [spirit, flesh, miasma]: ")

    # Capture the initial screen and do calibration

    # Bank boat
    interactor.activate()
    screenshot = interactor.capture()
    bank_boat_data = find_image(bank_boat_img, screenshot)
    if not bank_boat_data:
        print("Bank boat not found.")
        return
    x, y, w, h = bank_boat_data
    center_x, center_y = x + w // 2, y + h // 2
    bank_boat_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
    interactor.click(bank_boat_pos[0], bank_boat_pos[1], 3)
    time.sleep(random.uniform(0.6, 0.7))

    # Load last preset
    screenshot = interactor.capture()
    load_last_bank_preset_data = find_image(load_last_bank_preset_img, screenshot)
    x, y, w, h = load_last_bank_preset_data
    center_x, center_y = x + w // 2, y + h // 2
    load_last_bank_preset_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
    interactor.click(load_last_bank_preset_pos[0], load_last_bank_preset_pos[1], 1)
    time.sleep(random.uniform(0.6, 0.7))
    
    # Rub Passing Bracelet
    interactor.send_key('D')
    time.sleep(random.uniform(0.7, 0.9))

    # Haunt on the hill teleport
    screenshot = interactor.capture()
    haunt_on_the_hill_data = find_image(haunt_on_the_hill_img, screenshot)
    if haunt_on_the_hill_data:
        interactor.send_key('2')
        interactor.send_key('2')
    time.sleep(random.uniform(6.5, 7))
    
    # Presurge Darkportal location
    x,y,w,h = hoh_presurge_roi_data
    center_x, center_y = x + w // 2, y + h // 2
    dark_portal_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=4)
    interactor.click(dark_portal_presurge_pos[0], dark_portal_presurge_pos[1] , 1)
    time.sleep(random.uniform(1.8, 1.9))
    interactor.send_key(['Alt_L', '1'])
    time.sleep(random.uniform(1.4, 1.6))
    
    # Postsurge Darkportal location
    x, y, w, h = dark_portal_postsurge_data
    center_x, center_y = x + w // 2, y + h // 2
    dark_portal_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
    interactor.click(dark_portal_postsurge_pos[0], dark_portal_postsurge_pos[1] , 1)
    time.sleep(random.uniform(2.4, 2.8))

    if rune_type=='spirit':
        # Surge
        time.sleep(random.uniform(1.0, 1.1))
        interactor.send_key(['Alt_L', '1'])
        time.sleep(random.uniform(2, 2.4))
        
        # Find Spirit altar and craft runes
        screenshot = interactor.capture()
        spirit_altar_postsurge_data = find_image(spirit_altar_postsurge_img, screenshot, template_scales[bank_boat_img])
        x, y, w, h = spirit_altar_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        spirit_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)

        interactor.send_key('S')
        interactor.click(spirit_altar_postsurge_pos[0], spirit_altar_postsurge_pos[1] , 1)
        time.sleep(random.uniform(3.0, 3.25))

    if rune_type=='miasma':
        # Find Miasma altar and surge
        screenshot = interactor.capture()
        miasma_altar_presurge_data = find_image(miasma_altar_presurge_img, screenshot, template_scales[bank_boat_img])
        x, y, w, h = miasma_altar_presurge_data
        center_x, center_y = x + w // 2, y + h // 2
        miasma_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        interactor.click(miasma_altar_presurge_pos[0], miasma_altar_presurge_pos[1] , 1)
        time.sleep(random.uniform(0.8, 0.85))
        interactor.send_key(['Alt_L', '1'])
        time.sleep(random.uniform(2, 2.4))
        
        # Re-search Miasma altar and craft runes
        screenshot = interactor.capture()
        miasma_altar_postsurge_data = find_image(miasma_altar_postsurge_img, screenshot, template_scales[bank_boat_img])
        x, y, w, h = miasma_altar_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        miasma_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        interactor.send_key('S')
        interactor.click(miasma_altar_postsurge_pos[0], miasma_altar_postsurge_pos[1] , 1)
        time.sleep(random.uniform(1.15, 1.25))

    if rune_type=='flesh':
        # Find Flesh altar and surge
        screenshot = interactor.capture()
        flesh_altar_presurge_data = find_image(flesh_altar_presurge_img, screenshot, template_scales[bank_boat_img])
        x, y, w, h = flesh_altar_presurge_data
        w = w//2
        center_x, center_y = x + w // 2, y + h // 2
        flesh_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=4)
        interactor.click(flesh_altar_presurge_pos[0], flesh_altar_presurge_pos[1] , 1)
        time.sleep(random.uniform(1.2, 1.25))
        interactor.send_key(['Alt_L', '1'])
        time.sleep(random.uniform(2, 2.4))

        # Re-search Flesh altar and craft runes
        screenshot = interactor.capture()
        flesh_altar_postsurge_data = find_image(flesh_altar_postsurge_img, screenshot, template_scales[bank_boat_img])
        x, y, w, h = flesh_altar_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        flesh_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        interactor.send_key('S')
        interactor.click(flesh_altar_postsurge_pos[0], flesh_altar_postsurge_pos[1] , 1)
        time.sleep(random.uniform(1.6, 1.8))

    for _ in range(np.random.randint(4,6)):
        interactor.send_key('A')
    time.sleep(random.uniform(3.9, 4.1))
    
    iteration = 0

    while script_running:

        # Activate the window
        interactor.activate()
        if not script_running:
            break
        # Step 1 -> Right click bank boat
        x, y, w, h = bank_boat_data
        center_x, center_y = x + w // 2, y + h // 2
        bank_boat_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        interactor.click(bank_boat_pos[0], bank_boat_pos[1], 3)
        time.sleep(random.uniform(0.6, 0.7))
        
        if not script_running:
            break
        # Step 2 -> Find and left-click on "Load last bank preset"
        screenshot = interactor.capture()
        load_last_bank_preset_data = find_image(load_last_bank_preset_img, screenshot)
        x, y, w, h = load_last_bank_preset_data
        center_x, center_y = x + w // 2, y + h // 2
        load_last_bank_preset_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
        interactor.click(load_last_bank_preset_pos[0], load_last_bank_preset_pos[1], 1)
        time.sleep(random.uniform(0.6, 0.7))
        
        if not script_running:
            break
        # Step 3 -> Rub Passing Bracelet
        interactor.send_key('D')
        time.sleep(random.uniform(0.7, 0.9))

        if not script_running:
            break
        # Step 4 -> Haunt on the hill teleport
        screenshot = interactor.capture()
        haunt_on_the_hill_data = find_image(haunt_on_the_hill_img, screenshot)
        if haunt_on_the_hill_data:
            interactor.send_key('2')
            interactor.send_key('2')
        time.sleep(random.uniform(2.8, 3.0))

        if not script_running:
            break
        # Step 5 -> Surge to Dark Portal
        x,y,w,h = hoh_presurge_roi_data
        center_x, center_y = x + w // 2, y + h // 2
        dark_portal_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
        interactor.click(dark_portal_presurge_pos[0], dark_portal_presurge_pos[1] , 1)
        time.sleep(random.uniform(1.8, 1.9))
        interactor.send_key(['Alt_L', '1'])
        time.sleep(random.uniform(1.4, 1.6))

        if not script_running:
            break
        # Step 6 -> Enter the Dark Portal
        # Postsurge Darkportal location
        x, y, w, h = dark_portal_postsurge_data
        center_x, center_y = x + w // 2, y + h // 2
        dark_portal_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='circle', roi_diminish=3)
        interactor.click(dark_portal_postsurge_pos[0], dark_portal_postsurge_pos[1] , 1)
        time.sleep(random.uniform(1.2, 1.3))

        if not script_running:
            break
        if rune_type=='miasma':
            # Step 7 -> Find Miasma altar and surge
            x, y, w, h = miasma_altar_presurge_data
            center_x, center_y = x + w // 2, y + h // 2
            miasma_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            interactor.click(miasma_altar_presurge_pos[0], miasma_altar_presurge_pos[1] , 1)
            time.sleep(random.uniform(1.2, 1.25))
            interactor.send_key(['Alt_L', '1'])
            time.sleep(random.uniform(1.4, 1.5))

            if not script_running:
                break
            # Step 8 -> Re-search Miasma altar and craft runes
            x, y, w, h = miasma_altar_postsurge_data
            center_x, center_y = x + w // 2, y + h // 2
            miasma_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            interactor.send_key('S')
            interactor.click(flesh_altar_postsurge_pos[0], flesh_altar_postsurge_pos[1] , 1)
            time.sleep(random.uniform(1.6, 1.8))

        if rune_type=='flesh':
            # Step 7 -> Find Flesh altar and surge
            x, y, w, h = flesh_altar_presurge_data
            w = w//2
            center_x, center_y = x + w // 2, y + h // 2
            flesh_altar_presurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=4)
            interactor.click(flesh_altar_presurge_pos[0], flesh_altar_presurge_pos[1] , 1)
            time.sleep(random.uniform(1.2, 1.25))
            interactor.send_key(['Alt_L', '1'])
            time.sleep(random.uniform(1.4, 1.5))

            if not script_running:
                break
            # Step 8 -> Re-search flesh altar and craft runes
            x, y, w, h = flesh_altar_postsurge_data
            center_x, center_y = x + w // 2, y + h // 2
            flesh_altar_postsurge_pos = randomize_click_position(center_x, center_y, w, h, shape='rectangle', roi_diminish=2)
            interactor.send_key('S')
            interactor.click(flesh_altar_postsurge_pos[0], flesh_altar_postsurge_pos[1] , 1)
            time.sleep(random.uniform(1.6, 1.8))
            

        for _ in range(np.random.randint(4,6)):
            interactor.send_key('A')
        time.sleep(random.uniform(3.9, 4.1))
        
        if not script_running:
            break
        clear_output(wait=True)
        print(f"Crafted Iteration: {iteration}")
        iteration+=1
        if iteration == cycle_count:
            print("Cycles Completed")
            script_running = not script_running

In [ ]:
def on_press(key):
    global script_running
    if key == pkeyboard.KeyCode(char='-'):
        script_running = not script_running
        if script_running:
            print("Script started.")
            # Run the main script in a separate thread
            threading.Thread(target=main_script).start()
        else:
            print("Script stopped.")

In [ ]:
def start_listener():
    with pkeyboard.Listener(on_press=on_press) as listener:
        listener.join()

In [ ]:
start_listener()

In [ ]:
script_running = True
main_script()